In [ ]:
import nest_asyncio

nest_asyncio.apply()


Task2 완료!
Task1 완료!


In [ ]:
%autoawait


IPython autoawait is `on`, and set to use `asyncio`


In [ ]:
import asyncio


async def fetch_data(name, delay):
    await asyncio.sleep(delay)  # 논블로킹 I/O
    print(f"{name} 완료!")


async def main():
    await asyncio.gather(fetch_data("Task1", 2), fetch_data("Task2", 1))


await main()

<coroutine object main at 0x000001EF03CCB680>

In [6]:
import asyncio
import time


# 비동기 작업 정의
async def fetch_data(name, delay):
    await asyncio.sleep(delay)  # 논블로킹 I/O (delay 초 대기)
    print(f"{name} 완료!")


# main 함수
async def main():
    start = time.time()  # 시작 시간 기록

    await asyncio.gather(
        fetch_data("Task1", 2),  # 2초 지연
        fetch_data("Task2", 1),  # 1초 지연
    )

    end = time.time()  # 종료 시간 기록
    print(f"총 실행 시간: {end - start:.2f}초")


# 이벤트 루프 실행
asyncio.run(main())


Task2 완료!
Task1 완료!
총 실행 시간: 2.01초


In [9]:
import asyncio


async def worker(name, delay):
    await asyncio.sleep(delay)
    print(f"{name} 완료")
    return name


async def main():
    results = []
    async with asyncio.TaskGroup() as tg:
        task1 = tg.create_task(worker("작업1", 1))
        task2 = tg.create_task(worker("작업2", 2))
        task3 = tg.create_task(worker("작업3", 3))
        # 구조적으로 관리되므로 스코프 종료 시 자동 취소/정리

    results.extend([task1.result(), task2.result(), task3.result()])
    print("모든 작업 완료:", results)


await main()


작업1 완료
작업2 완료
작업3 완료
모든 작업 완료: ['작업1', '작업2', '작업3']


In [10]:
import requests
import threading


def fetch(url, callback):
    def task():
        try:
            resp = requests.get(url)
            callback(None, resp.text)
        except Exception as e:
            callback(e, None)

    threading.Thread(target=task).start()


results = []


def step1(err, data):
    if err:
        print("Error:", err)
        return
    results.append(data)
    fetch("https://httpbin.org/get", step2)


def step2(err, data):
    if err:
        print("Error:", err)
        return
    results.append(data)
    fetch("https://httpbin.org/get", step3)


def step3(err, data):
    if err:
        print("Error:", err)
        return
    results.append(data)
    print("✅ Callback 결과:", len(results))


fetch("https://httpbin.org/get", step1)

✅ Callback 결과: 3


In [11]:
results


['{\n  "args": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.32.3", \n    "X-Amzn-Trace-Id": "Root=1-68afca3e-5220593d735814b46af2e23a"\n  }, \n  "origin": "112.171.20.239", \n  "url": "https://httpbin.org/get"\n}\n',
 '{\n  "args": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.32.3", \n    "X-Amzn-Trace-Id": "Root=1-68afca3f-1072c1e206d8353f53f7a6e8"\n  }, \n  "origin": "112.171.20.239", \n  "url": "https://httpbin.org/get"\n}\n',
 '{\n  "args": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.32.3", \n    "X-Amzn-Trace-Id": "Root=1-68afca40-63a6116c12f69a960ad009f2"\n  }, \n  "origin": "112.171.20.239", \n  "url": "https://httpbin.org/get"\n}\n']

In [12]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests

urls = ["https://httpbin.org/get"] * 3


def fetch(url):
    return requests.get(url).text


with ThreadPoolExecutor() as executor:
    futures = [executor.submit(fetch, url) for url in urls]
    results = []
    for future in as_completed(futures):
        try:
            results.append(future.result())
        except Exception as e:
            print("Error:", e)

print("✅ Future 결과:", len(results))

✅ Future 결과: 3


In [13]:
results


['{\n  "args": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.32.3", \n    "X-Amzn-Trace-Id": "Root=1-68afca7c-47aeca4e17f8fd6117daacb5"\n  }, \n  "origin": "112.171.20.239", \n  "url": "https://httpbin.org/get"\n}\n',
 '{\n  "args": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.32.3", \n    "X-Amzn-Trace-Id": "Root=1-68afca7c-32fa66dc102da121274f6c35"\n  }, \n  "origin": "112.171.20.239", \n  "url": "https://httpbin.org/get"\n}\n',
 '{\n  "args": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.32.3", \n    "X-Amzn-Trace-Id": "Root=1-68afca7c-1c6f3c2f0d62817162159d5b"\n  }, \n  "origin": "112.171.20.239", \n  "url": "https://httpbin.org/get"\n}\n']

In [15]:
import aiohttp
import asyncio

urls = ["https://httpbin.org/get"] * 3


async def fetch(session, url):
    async with session.get(url) as resp:
        return await resp.text()


async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        results = await asyncio.gather(*tasks, return_exceptions=True)

        # 에러 필터링
        results = [r for r in results if not isinstance(r, Exception)]
        print("✅ Async/Await 결과:", len(results))


await main()

✅ Async/Await 결과: 3


In [16]:
import aiohttp
import asyncio

urls = ["https://httpbin.org/get"] * 3


async def fetch(session, url):
    async with session.get(url) as resp:
        return await resp.text()


async def main():
    results = []
    async with aiohttp.ClientSession() as session:
        async with asyncio.TaskGroup() as tg:
            tasks = [tg.create_task(fetch(session, url)) for url in urls]

        # 모든 TaskGroup 태스크는 자동 취소·정리됨
        for task in tasks:
            if not task.exception():
                results.append(task.result())

    print("✅ 구조적 동시성 결과:", len(results))


await main()

✅ 구조적 동시성 결과: 3


In [18]:
# traffic_analyzer.py - 메인 분석 시스템
import asyncio
import time
import json
import logging
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Dict, List, Optional
from collections import deque, defaultdict

import numpy as np
import pandas as pd
from influxdb_client import InfluxDBClient, Point
from pysnmp.hlapi import *
import websockets
from fastapi import FastAPI, WebSocket
import uvicorn

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


@dataclass
class TrafficMetrics:
    """트래픽 메트릭 데이터 클래스"""

    timestamp: datetime
    interface: str
    host: str
    bytes_in: int
    bytes_out: int
    packets_in: int
    packets_out: int
    utilization_percent: float
    errors_in: int = 0
    errors_out: int = 0


class TrafficCollector:
    """네트워크 트래픽 수집기 - SNMP 기반"""

    def __init__(self, hosts: List[str], community: str = "public"):
        self.hosts = hosts
        self.community = community
        # 이전 수집 데이터 저장 (델타 계산용)
        self.previous_data = {}

        # SNMP OID 정의
        self.oids = {
            "ifDescr": "1.3.6.1.2.1.2.2.1.2",  # 인터페이스 설명
            "ifSpeed": "1.3.6.1.2.1.2.2.1.5",  # 인터페이스 속도
            "ifInOctets": "1.3.6.1.2.1.2.2.1.10",  # 수신 바이트
            "ifOutOctets": "1.3.6.1.2.1.2.2.1.16",  # 송신 바이트
            "ifInUcastPkts": "1.3.6.1.2.1.2.2.1.11",  # 수신 패킷
            "ifOutUcastPkts": "1.3.6.1.2.1.2.2.1.17",  # 송신 패킷
            "ifInErrors": "1.3.6.1.2.1.2.2.1.14",  # 수신 에러
            "ifOutErrors": "1.3.6.1.2.1.2.2.1.20",  # 송신 에러
        }

    async def discover_interfaces(self, host: str) -> Dict[int, Dict]:
        """네트워크 장비의 인터페이스 자동 발견"""
        interfaces = {}

        # 인터페이스 설명과 속도 수집
        for errorIndication, errorStatus, errorIndex, varBinds in nextCmd(
            SnmpEngine(),
            CommunityData(self.community),
            UdpTransportTarget((host, 161)),
            ContextData(),
            ObjectType(ObjectIdentity(self.oids["ifDescr"])),
            lexicographicMode=False,
        ):
            if errorIndication:
                logger.error(f"SNMP 에러: {errorIndication}")
                break

            for varBind in varBinds:
                oid_str = str(varBind[0])
                if self.oids["ifDescr"] in oid_str:
                    # 인터페이스 인덱스 추출
                    if_index = int(oid_str.split(".")[-1])
                    if_descr = str(varBind[1])

                    # Loopback이나 관리 인터페이스 제외
                    if not any(
                        skip in if_descr.lower()
                        for skip in ["loopback", "null", "management"]
                    ):
                        interfaces[if_index] = {
                            "description": if_descr,
                            "speed": await self.get_interface_speed(host, if_index),
                        }

        logger.info(f"{host}에서 {len(interfaces)}개 인터페이스 발견")
        return interfaces

    async def get_interface_speed(self, host: str, if_index: int) -> int:
        """인터페이스 속도 조회 (bps 단위)"""
        oid = f"{self.oids['ifSpeed']}.{if_index}"

        for errorIndication, errorStatus, errorIndex, varBinds in getCmd(
            SnmpEngine(),
            CommunityData(self.community),
            UdpTransportTarget((host, 161)),
            ContextData(),
            ObjectType(ObjectIdentity(oid)),
        ):
            if errorIndication:
                return 1000000000  # 기본값 1Gbps

            for varBind in varBinds:
                speed = int(varBind[1])
                # 32bit 제한으로 인한 높은 속도 인터페이스 처리
                if speed == 4294967295:  # 2^32-1
                    # ifHighSpeed OID 사용 (Mbps 단위)
                    high_speed_oid = f"1.3.6.1.2.1.2.2.1.31.1.1.1.15.{if_index}"
                    # 구현 단순화를 위해 10Gbps로 가정
                    return 10000000000
                return speed

        return 1000000000  # 기본값

    async def collect_interface_data(
        self, host: str, if_index: int, if_info: Dict
    ) -> Optional[TrafficMetrics]:
        """특정 인터페이스의 트래픽 데이터 수집"""
        current_data = {}

        # 각 메트릭에 대해 SNMP 조회
        for metric, base_oid in self.oids.items():
            if metric in ["ifDescr", "ifSpeed"]:
                continue  # 이미 수집됨

            oid = f"{base_oid}.{if_index}"

            for errorIndication, errorStatus, errorIndex, varBinds in getCmd(
                SnmpEngine(),
                CommunityData(self.community),
                UdpTransportTarget((host, 161)),
                ContextData(),
                ObjectType(ObjectIdentity(oid)),
            ):
                if errorIndication:
                    logger.warning(f"메트릭 {metric} 수집 실패: {errorIndication}")
                    return None

                for varBind in varBinds:
                    current_data[metric] = int(varBind[1])

        # 이전 데이터와 비교하여 델타 계산
        key = f"{host}:{if_index}"
        timestamp = datetime.now()

        if key in self.previous_data:
            prev_data, prev_time = self.previous_data[key]
            time_delta = (timestamp - prev_time).total_seconds()

            if time_delta > 0:
                # 카운터 차이 계산 (32bit 오버플로우 고려)
                MAX_COUNTER = 2**32

                bytes_in_delta = self._calculate_counter_delta(
                    current_data["ifInOctets"], prev_data["ifInOctets"], MAX_COUNTER
                )
                bytes_out_delta = self._calculate_counter_delta(
                    current_data["ifOutOctets"], prev_data["ifOutOctets"], MAX_COUNTER
                )

                # 초당 전송률 계산
                bytes_in_per_sec = bytes_in_delta / time_delta
                bytes_out_per_sec = bytes_out_delta / time_delta

                # 대역폭 사용률 계산
                max_throughput_bps = max(bytes_in_per_sec * 8, bytes_out_per_sec * 8)
                utilization = (
                    (max_throughput_bps / if_info["speed"]) * 100
                    if if_info["speed"] > 0
                    else 0
                )

                # 트래픽 메트릭 객체 생성
                metrics = TrafficMetrics(
                    timestamp=timestamp,
                    interface=if_info["description"],
                    host=host,
                    bytes_in=int(bytes_in_per_sec),
                    bytes_out=int(bytes_out_per_sec),
                    packets_in=self._calculate_counter_delta(
                        current_data["ifInUcastPkts"],
                        prev_data["ifInUcastPkts"],
                        MAX_COUNTER,
                    )
                    // int(time_delta),
                    packets_out=self._calculate_counter_delta(
                        current_data["ifOutUcastPkts"],
                        prev_data["ifOutUcastPkts"],
                        MAX_COUNTER,
                    )
                    // int(time_delta),
                    utilization_percent=min(utilization, 100),  # 100% 제한
                    errors_in=current_data.get("ifInErrors", 0),
                    errors_out=current_data.get("ifOutErrors", 0),
                )

                # 이전 데이터 업데이트
                self.previous_data[key] = (current_data, timestamp)
                return metrics

        # 첫 수집 시 데이터만 저장
        self.previous_data[key] = (current_data, timestamp)
        return None

    def _calculate_counter_delta(
        self, current: int, previous: int, max_value: int
    ) -> int:
        """SNMP 카운터의 델타값 계산 (오버플로우 고려)"""
        if current >= previous:
            return current - previous
        else:
            # 카운터 롤오버 발생
            return (max_value - previous) + current


class AnomalyDetector:
    """트래픽 이상 탐지기 - 통계 기반"""

    def __init__(self, window_size: int = 288):  # 24시간 = 288 * 5분
        self.window_size = window_size
        # 인터페이스별 히스토리 데이터 저장
        self.history = defaultdict(lambda: deque(maxlen=window_size))
        self.baseline_stats = {}

    def update_baseline(self, interface: str, utilization: float):
        """기준선 통계 업데이트"""
        self.history[interface].append(utilization)

        if len(self.history[interface]) >= 30:  # 최소 30개 샘플
            data = np.array(list(self.history[interface]))
            self.baseline_stats[interface] = {
                "mean": np.mean(data),
                "std": np.std(data),
                "median": np.median(data),
                "p95": np.percentile(data, 95),
                "p99": np.percentile(data, 99),
            }

    def detect_anomaly(self, interface: str, current_utilization: float) -> Dict:
        """이상 탐지 수행"""
        if interface not in self.baseline_stats:
            return {"anomaly": False, "reason": "기준선 부족"}

        stats = self.baseline_stats[interface]

        # 1. Z-score 기반 이상 탐지
        if stats["std"] > 0:
            z_score = (current_utilization - stats["mean"]) / stats["std"]
        else:
            z_score = 0

        # 2. 백분위 기반 이상 탐지
        percentile_anomaly = current_utilization > stats["p99"]

        # 3. 급격한 변화 탐지
        recent_data = list(self.history[interface])[-5:]  # 최근 5개 값
        if len(recent_data) >= 2:
            recent_change = abs(current_utilization - recent_data[-2])
            rapid_change = recent_change > (2 * stats["std"])
        else:
            rapid_change = False

        # 종합 판정
        anomaly_detected = abs(z_score) > 3 or percentile_anomaly or rapid_change

        return {
            "anomaly": anomaly_detected,
            "severity": "high"
            if abs(z_score) > 4
            else "medium"
            if abs(z_score) > 3
            else "low",
            "z_score": z_score,
            "current_value": current_utilization,
            "expected_range": (
                stats["mean"] - 2 * stats["std"],
                stats["mean"] + 2 * stats["std"],
            ),
            "percentile_exceeded": percentile_anomaly,
            "rapid_change": rapid_change,
        }


class TrafficAnalysisSystem:
    """통합 트래픽 분석 시스템"""

    def __init__(
        self,
        influxdb_url: str,
        influxdb_token: str,
        influxdb_org: str,
        influxdb_bucket: str,
    ):
        # InfluxDB 클라이언트 초기화
        self.influx_client = InfluxDBClient(
            url=influxdb_url, token=influxdb_token, org=influxdb_org
        )
        self.write_api = self.influx_client.write_api()
        self.query_api = self.influx_client.query_api()
        self.bucket = influxdb_bucket

        # 컴포넌트 초기화
        self.collector = TrafficCollector(
            ["192.168.1.1", "10.0.0.1"]
        )  # 실제 IP로 변경 필요
        self.anomaly_detector = AnomalyDetector()

        # WebSocket 연결 관리
        self.websocket_connections = set()

        # FastAPI 앱 초기화
        self.app = FastAPI()
        self.setup_routes()

    def setup_routes(self):
        """FastAPI 라우트 설정"""

        @self.app.websocket("/ws")
        async def websocket_endpoint(websocket: WebSocket):
            await websocket.accept()
            self.websocket_connections.add(websocket)
            try:
                while True:
                    await websocket.receive_text()  # 연결 유지
            except:
                self.websocket_connections.remove(websocket)

        @self.app.get("/api/interfaces/{host}")
        async def get_interfaces(host: str):
            """특정 호스트의 인터페이스 목록 조회"""
            try:
                interfaces = await self.collector.discover_interfaces(host)
                return {"success": True, "interfaces": interfaces}
            except Exception as e:
                return {"success": False, "error": str(e)}

        @self.app.get("/api/traffic/summary")
        async def get_traffic_summary():
            """전체 트래픽 요약 통계"""
            query = f"""
            from(bucket: "{self.bucket}")
            |> range(start: -1h)
            |> filter(fn: (r) => r._measurement == "network_traffic")
            |> filter(fn: (r) => r._field == "utilization_percent")
            |> group(columns: ["host", "interface"])
            |> aggregateWindow(every: 5m, fn: mean)
            |> yield(name: "mean")
            """

            try:
                result = self.query_api.query(query)
                summary = []
                for table in result:
                    for record in table.records:
                        summary.append(
                            {
                                "host": record.values.get("host"),
                                "interface": record.values.get("interface"),
                                "avg_utilization": record.get_value(),
                                "timestamp": record.get_time().isoformat(),
                            }
                        )
                return {"success": True, "data": summary}
            except Exception as e:
                logger.error(f"트래픽 요약 조회 오류: {e}")
                return {"success": False, "error": str(e)}

    async def store_metrics(self, metrics: TrafficMetrics):
        """메트릭을 InfluxDB에 저장"""
        point = (
            Point("network_traffic")
            .tag("host", metrics.host)
            .tag("interface", metrics.interface)
            .field("bytes_in", metrics.bytes_in)
            .field("bytes_out", metrics.bytes_out)
            .field("packets_in", metrics.packets_in)
            .field("packets_out", metrics.packets_out)
            .field("utilization_percent", metrics.utilization_percent)
            .field("errors_in", metrics.errors_in)
            .field("errors_out", metrics.errors_out)
            .time(metrics.timestamp)
        )

        self.write_api.write(bucket=self.bucket, record=point)

    async def broadcast_real_time_data(self, data: Dict):
        """실시간 데이터를 WebSocket으로 브로드캐스트"""
        if self.websocket_connections:
            message = json.dumps(data)
            disconnected = set()

            for websocket in self.websocket_connections:
                try:
                    await websocket.send_text(message)
                except:
                    disconnected.add(websocket)

            # 연결 끊긴 WebSocket 제거
            self.websocket_connections -= disconnected

    async def analyze_traffic(self):
        """주기적 트래픽 분석 및 이상 탐지"""
        logger.info("트래픽 분석 시작")

        while True:
            try:
                for host in self.collector.hosts:
                    # 인터페이스 발견
                    interfaces = await self.collector.discover_interfaces(host)

                    for if_index, if_info in interfaces.items():
                        # 트래픽 데이터 수집
                        metrics = await self.collector.collect_interface_data(
                            host, if_index, if_info
                        )

                        if metrics:
                            # 데이터베이스 저장
                            await self.store_metrics(metrics)

                            # 이상 탐지
                            self.anomaly_detector.update_baseline(
                                f"{metrics.host}:{metrics.interface}",
                                metrics.utilization_percent,
                            )

                            anomaly_result = self.anomaly_detector.detect_anomaly(
                                f"{metrics.host}:{metrics.interface}",
                                metrics.utilization_percent,
                            )

                            # 실시간 데이터 브로드캐스트
                            real_time_data = {
                                "timestamp": metrics.timestamp.isoformat(),
                                "host": metrics.host,
                                "interface": metrics.interface,
                                "utilization": metrics.utilization_percent,
                                "throughput_mbps": (
                                    metrics.bytes_in + metrics.bytes_out
                                )
                                * 8
                                / 1000000,
                                "anomaly": anomaly_result,
                            }

                            await self.broadcast_real_time_data(real_time_data)

                            # 이상 탐지 시 알림
                            if anomaly_result["anomaly"]:
                                logger.warning(
                                    f"트래픽 이상 탐지: {metrics.host}:{metrics.interface} - "
                                    f"사용률 {metrics.utilization_percent:.1f}%, "
                                    f"Z-score: {anomaly_result['z_score']:.2f}"
                                )

            except Exception as e:
                logger.error(f"트래픽 분석 중 오류: {e}")

            # 5분 간격으로 분석
            await asyncio.sleep(300)

    async def start(self):
        """시스템 시작"""
        logger.info("네트워크 트래픽 분석 시스템 시작")

        # 백그라운드 태스크로 트래픽 분석 시작
        analysis_task = asyncio.create_task(self.analyze_traffic())

        # FastAPI 서버 시작
        config = uvicorn.Config(
            app=self.app, host="0.0.0.0", port=8000, log_level="info"
        )
        server = uvicorn.Server(config)
        await server.serve()


# 메인 실행 함수
async def main():
    # InfluxDB 설정 (실제 값으로 변경 필요)
    INFLUXDB_URL = "http://localhost:8086"
    INFLUXDB_TOKEN = "your-influxdb-token"
    INFLUXDB_ORG = "your-org"
    INFLUXDB_BUCKET = "network-traffic"

    # 분석 시스템 초기화 및 시작
    system = TrafficAnalysisSystem(
        INFLUXDB_URL, INFLUXDB_TOKEN, INFLUXDB_ORG, INFLUXDB_BUCKET
    )
    await system.start()


if __name__ == "__main__":
    asyncio.run(main())

C:\Users\hy\AppData\Local\Programs\Python\Python312\Lib\collections\__init__.py:447: RuntimeWarning: coroutine 'main' was never awaited
  @classmethod


ModuleNotFoundError: No module named 'pysnmp'